# Project 7

## Sys Ops

In [1]:
cd ..

C:\Users\rudyg\Documents\GitHub\P7-ML


In [2]:
pwd

'C:\\Users\\rudyg\\Documents\\GitHub\\P7-ML'

## Imports 

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import missingno as msno

from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.decomposition import *
from sklearn.model_selection import *
from sklearn.impute import *
from sklearn.preprocessing import *
from sklearn.pipeline import *
from sklearn.covariance import *
from sklearn.metrics import *
from sklearn.compose import *

from sklearn.dummy import *
from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.ensemble import *
from sklearn.cluster import *


from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [4]:
# DO NOT REMOVE THIS LINE

## Data

In [5]:
fn = "df.csv"
df = pd.read_csv("./data/preprocessed/" + fn)

In [6]:
df.shape

(307454, 20)

In [7]:
df = df.sample(frac=0.10)
df

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,EXT_SOURCE_2,EXT_SOURCE_3,nan_EXT_SOURCE_1
226079,0,Cash loans,1,1,0,0,247500.0,450000.0,27193.5,450000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,38.800000,7.498630,0.285898,NaN,0
177382,0,Cash loans,2,0,0,1,135000.0,727785.0,19327.5,607500.0,Other_B,Working,Secondary / secondary special,Single / not married,House / apartment,35.780822,6.589041,0.737190,0.298595,0
93848,0,Cash loans,1,1,1,0,180000.0,157914.0,18868.5,139500.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,27.583562,5.643836,0.636314,NaN,1
28405,1,Cash loans,1,1,1,0,157500.0,512064.0,19431.0,360000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,47.364384,2.936986,0.643394,0.468660,0
160013,0,Cash loans,2,0,1,0,112500.0,675000.0,19476.0,675000.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,61.463014,NaN,0.639622,0.703203,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182702,0,Cash loans,1,1,1,3,270000.0,840996.0,24718.5,702000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,35.600000,7.715068,0.547302,0.431192,1
35563,0,Cash loans,2,0,0,0,135000.0,337500.0,16546.5,337500.0,Unaccompanied,Working,Higher education,Separated,House / apartment,37.589041,15.353425,0.656621,0.646330,0
305369,0,Cash loans,2,0,1,0,157500.0,417915.0,23463.0,378000.0,Unaccompanied,Working,Higher education,Married,House / apartment,39.904110,13.027397,0.493486,0.588488,0
108475,0,Cash loans,2,1,0,0,225000.0,900000.0,32458.5,900000.0,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,48.095890,12.673973,0.569176,0.429424,0


In [8]:
X = df.drop(columns=["TARGET"])
y  = df["TARGET"]

In [9]:
y.value_counts(normalize=True).round(2)

TARGET
0    0.92
1    0.08
Name: proportion, dtype: float64

In [10]:
def resultize(grid, verbose=2):
    """make results based on gridsearchcv"""

    # make df
    results = pd.DataFrame(grid.cv_results_)

    # clean columns
    cols = [i for i in results.columns if not i.startswith("split")]
    results = results.loc[:, cols]

    if verbose == 2:
        print("coucou")

    return results.round(2)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [12]:
y_train.value_counts(normalize=True)

TARGET
0    0.920895
1    0.079105
Name: proportion, dtype: float64

In [13]:
y_test.value_counts(normalize=True)

TARGET
0    0.918043
1    0.081957
Name: proportion, dtype: float64

## Modelisation

### Refactoring avec utilisation de Pipeline 

### Création du ColumnTransformer

In [14]:
numerical_features = [
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "CNT_CHILDREN",
    "AMT_INCOME_TOTAL",
    "AMT_CREDIT",
    "AMT_ANNUITY",
    "AMT_GOODS_PRICE",
    "DAYS_BIRTH",
    "DAYS_EMPLOYED",
    # "EXT_SOURCE_1",
    "EXT_SOURCE_2",
    "EXT_SOURCE_3",
]

categorical_features = [
    "NAME_CONTRACT_TYPE",
    "NAME_TYPE_SUITE",
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    # "OCCUPATION_TYPE",
    # "ORGANIZATION_TYPE",
]

In [15]:
# Création de 2 Pipelines

numerical_pipeline = make_pipeline(KNNImputer(), StandardScaler())

categorical_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OneHotEncoder()
)

In [16]:
# Création du ColumnTransformer
# numerical_pipeline est appliquée sur les numerical_features
# categorical_pipeline est appliquée sur les categorical_features

ColumnTransformer = make_column_transformer(
    (numerical_pipeline, numerical_features),
    (categorical_pipeline, categorical_features),
)
ColumnTransformer

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('knnimputer', KNNImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['CODE_GENDER', 'FLAG_OWN_CAR',
                                  'FLAG_OWN_REALTY', 'CNT_CHILDREN',
                                  'AMT_INCOME_TOTAL', 'AMT_CREDIT',
                                  'AMT_ANNUITY', 'AMT_GOODS_PRICE',
                                  'DAYS_BIRTH', 'DAYS_EMPLOYED', 'EXT_SOURCE_2',
                                  'EXT_SOURCE_3']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['NAME_CONTRACT_TYPE', 'NAME_TYPE_SUITE',
                                  'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                                  'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE'])])

In [17]:


pipe = Pipeline(
    [
        ("columntransformer", ColumnTransformer),
        #  ("inputer", KNNImputer()),
        #  ("scaler", StandardScaler()),
        ('under_sampler', RandomUnderSampler()),
        ("estimator", DummyClassifier()),
    ]
)

In [18]:
param_grid = {
    "estimator": [
        DummyClassifier(),
        LogisticRegression(),
        RandomForestClassifier(),
    ],
}

In [19]:
grid2 = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=5,
    verbose=2,
    return_train_score=True,
    n_jobs=-1,
    error_score='raise'
)
grid2

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['CODE_GENDER',
                                                                          'FLAG_OWN_CAR',
                                                                          'FLAG_OWN_REALTY',
                                                                          'CNT_CHILDREN',
                                                                          'AMT_INCOME_TOTAL',
                                                                          'AMT_CREDIT',
                                                                          'AMT_ANNUITY',
                                                                          'AMT_GOODS_PRICE',
                                                                          'DAYS_BIRT...
                                                                                          OneHotEncoder())]),
                                                                         ['NAME_CONTRACT_TYPE',
                                                                          'NAME_TYPE_SUITE',
                                                                          'NAME_INCOME_TYPE',
                                                                          'NAME_EDUCATION_TYPE',
                                                                          'NAME_FAMILY_STATUS',
                                                                          'NAME_HOUSING_TYPE'])])),
                                       ('under_sampler', RandomUnderSampler()),
                                       ('estimator', DummyClassifier())]),
             n_jobs=-1,
             param_grid={'estimator': [DummyClassifier(), LogisticRegression(),
                                       RandomForestClassifier()]},
             return_train_score=True, verbose=2)

In [20]:
grid2.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('knnimputer',
                                                                                          KNNImputer()),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['CODE_GENDER',
                                                                          'FLAG_OWN_CAR',
                                                                          'FLAG_OWN_REALTY',
                                                                          'CNT_CHILDREN',
                                                                          'AMT_INCOME_TOTAL',
                                                                          'AMT_CREDIT',
                                                                          'AMT_ANNUITY',
                                                                          'AMT_GOODS_PRICE',
                                                                          'DAYS_BIRT...
                                                                                          OneHotEncoder())]),
                                                                         ['NAME_CONTRACT_TYPE',
                                                                          'NAME_TYPE_SUITE',
                                                                          'NAME_INCOME_TYPE',
                                                                          'NAME_EDUCATION_TYPE',
                                                                          'NAME_FAMILY_STATUS',
                                                                          'NAME_HOUSING_TYPE'])])),
                                       ('under_sampler', RandomUnderSampler()),
                                       ('estimator', DummyClassifier())]),
             n_jobs=-1,
             param_grid={'estimator': [DummyClassifier(), LogisticRegression(),
                                       RandomForestClassifier()]},
             return_train_score=True, verbose=2)

In [21]:
resultize(grid2)

coucou


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
0,21.15,2.57,3.79,0.36,DummyClassifier(),{'estimator': DummyClassifier()},0.92,0.0,1,0.92,0.00
1,17.70,0.39,3.80,0.39,LogisticRegression(),{'estimator': LogisticRegression()},0.68,0.0,2,0.68,0.00
2,18.74,1.42,3.59,0.48,RandomForestClassifier(),{'estimator': RandomForestClassifier()},0.66,0.0,3,0.71,0.01


In [22]:
grid2.cv_results_

{'mean_fit_time': array([21.14971313, 17.6974247 , 18.7361989 ]),
 'std_fit_time': array([2.57314815, 0.38586622, 1.42498211]),
 'mean_score_time': array([3.79233255, 3.80470738, 3.58708129]),
 'std_score_time': array([0.35815465, 0.38551585, 0.47878363]),
 'param_estimator': masked_array(data=[DummyClassifier(), LogisticRegression(),
                    RandomForestClassifier()],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'estimator': DummyClassifier()},
  {'estimator': LogisticRegression()},
  {'estimator': RandomForestClassifier()}],
 'split0_test_score': array([0.92085863, 0.68061578, 0.66240243]),
 'split1_test_score': array([0.92085863, 0.67627927, 0.65936687]),
 'split2_test_score': array([0.92085863, 0.67758023, 0.65611448]),
 'split3_test_score': array([0.92105834, 0.6751247 , 0.65300369]),
 'split4_test_score': array([0.92084147, 0.67859466, 0.66536543]),
 'mean_test_score': array([0.92089514, 0.67763893, 0.65925